#  LMS Exploration | Lyricist Language Model
***

**[Introduce]**  
* Language Model RNN(순환신경망) 개념을 이용해서 Lyricist AI 를 만들어보자 

* Lyricist AI는 노래의 가사를 학습해서 스스로 노래 가사 문장을 생성해내는 언어 모델 인공지능이다.
  

* 언어모델은 n-1개의 단어 시퀀스가 주어졌을 때, $n$번째 단어 $w_n$으로 무엇이 올지를 예측하는 확률모델이다.

* 모델에는 1개의 Embedding Layer, 2개의 LSTM Layer, 1개의 Dense Layer 로 구성된다.

**[dataset]**
* 영문 노래 가사가 적혀져 있는 49개의 파일을 사용함    
* 총 187088 개의 문장이 포함됨  
* 토큰의 개수가 15개를 넘어가는 문장은 학습 데이터에서 제외했음  
* 데이터 전처리 후 총 데이터의 20%를 평가 데이터셋으로 사용함  

**[Prepartion]**  

* dataset으로 사용할 파일들을 업로드하기

---

## 1. 데이터 읽어오기

* glob.glob() : 유닉스 스타일 경로명 패턴 확장 -> lyrics 폴더 안의 약 49개 파일의 경로이름 리스트 생성

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
print(f'txt_file_path에 속한 총 파일의 개수는 {len(txt_list)} 입니다.\n')
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("\nExamples:\n", raw_corpus[:3])

txt_file_path에 속한 총 파일의 개수는 49 입니다.

데이터 크기: 187088

Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   #- 길이가 0인 문장 건너뛰기
    if sentence[-1] == ":": continue  #- 문장의 끝이 : 인 문장 건너뛰기

    if idx > 15: break   #- 우선 문장 15개 확인
        
    print(sentence)

At first I was afraid
I was petrified
I kept thinking I could never live without you
By my side But then I spent so many nights
Just thinking how you've done me wrong
I grew strong
I learned how to get along And so you're back
From outer space
I just walked in to find you
Here without that look upon your face I should have changed that fucking lock
I would have made you leave your key
If I had known for just one second
You'd be back to bother me Well now go,
Walk out the door
Just turn around
Now, you're not welcome anymore Weren't you the one


---

## 2. 데이터 전처리 | Data Preprocess

### 2-1. 문장 정제하기
* 정규식(Regex)를 이용해 데이터를 정제한다.

In [3]:
import re
def preprocess_sentence(sentence): #- 정규표현식을 이용한 필터링 함수
    sentence = sentence.lower().strip() #- 소문자로 바꾸고, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #- 특수문자의 양쪽에 공백을 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) #- 여러개의 공백은 하나의 공백으로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #- a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
    sentence = sentence.strip() #- 양쪽 공백 지우기 
    sentence = '<start> ' + sentence + ' <end>' #- 문장의 시작에는 <start>, 끝에는 <end> 추가
    return sentence

corpus = []

for sentence in raw_corpus: #- raw_corpus 는 문장 리스트? 
    if len(sentence) == 0 : continue #- 길이가 0인 문장은 건너뜀 
    
    preprocessed_sentence = preprocess_sentence(sentence) 
    corpus.append(preprocessed_sentence) #- corpus 리스트에는 필터링된 문장들이 포함됨

corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

### 2-2. 데이터의 토큰화
* 텐서플로우는 자연어 처리를 위한 여러가지 모듈을 제공함.
* tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어사전을 만들어주며, 데이터를 숫자로 변환해줌
* 이처럼 데이터를 숫자로 변환하는 것을 벡터화(vetorize)라고 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭함
* 텐서플로우로 만든 모델의 입출력 데이터는 실제로 이러한 **텐서**로 변환되어 처리되는 것

In [4]:
import tensorflow as tf
import numpy as np


def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  #- num_words 를 단어 빈도수가 높은 15000개만 사용함
        filters=' ',
        oov_token="<unk>"
    )
    
    list_corpus = tokenizer.fit_on_texts(corpus) #- 문자 데이터를 입력받아서 리스트의 형태로 변환 
    tensor = tokenizer.texts_to_sequences(corpus) #- 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환
    print(f'정규식을 이용한 필터링 후의 문장의 개수는 총 {len(tensor)}개 입니다.')
    
    a = []
    for i in tensor:
        if len(i) <= 15:
            a.append(i)
        tensor = a
    print(f'이후 token의 개수가 15개를 초과하는 문장을 배제하여, 우리가 사용할 문장의 개수는 총 {len(tensor)}개 입니다')

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') #- 숫자 0을 이용해 같은 길이의 시퀀스로 변환 
    
    return tensor, tokenizer


tensor, tokenizer = tokenize(corpus)

정규식을 이용한 필터링 후의 문장의 개수는 총 175986개 입니다.
이후 token의 개수가 15개를 초과하는 문장을 배제하여, 우리가 사용할 문장의 개수는 총 156227개 입니다


### 2-3. 텐서를 소스와 타겟으로 분리

* **소스문장(Source Sentence)** : 자연어처리 분야에서 모델의 입력이 되는 문장
* **타겟 문장(Target Sentence)** : 정답 역할을 하게 될 모델의 출력 문장

In [5]:
for idx in tokenizer.index_word: #- tokenizer.index_word를 이용하여 각 토큰의 인덱스를 확인
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

src_input = tensor[:, :-1] #- tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성  
tgt_input = tensor[:, 1:] #- tensor에서 <start>를 잘라내서 타겟 문장을 생성

print(src_input[0]) #- 소스는 2(<start>)에서 시작해서 3(<end>)으로 끝난 후 0(<pad>)로 채워짐
print(tgt_input[0]) #- 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가지고 있음 

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
[  2  71 241   5  57 665   3   0   0   0   0   0   0   0]
[ 71 241   5  57 665   3   0   0   0   0   0   0   0   0]


### 2-4. 데이터셋 객체 생성하기
* 텐서플로우는 텐서로 생성된 데이터를 이용해 tf.data.Dataset 객체를 생성하는 방법을 흔히 사용함

In [6]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) #- tf.data.Dataset객체를 생성
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

---

## 3. 인공지능 학습시키기
* tf.keras.Model을 Subclassing하는 방식을 선택
* 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성됨

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2, random_state=1234)

In [8]:
print("Source Train:", np.shape(enc_train))
print("Target Train:", np.shape(dec_train))

Source Train: (124981, 14)
Target Train: (124981, 14)


In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* Embedding Layer의 역할  
    * 입력 tensor에 들어있는 단어사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔줌
    * 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용됨

In [10]:
optimizer = tf.keras.optimizers.Adam()

In [11]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
610/610 [==============================] - 220s 352ms/step - loss: 3.9733
Epoch 2/10
610/610 [==============================] - 218s 357ms/step - loss: 3.0213
Epoch 3/10
610/610 [==============================] - 220s 360ms/step - loss: 2.8398
Epoch 4/10
610/610 [==============================] - 221s 361ms/step - loss: 2.7112
Epoch 5/10
610/610 [==============================] - 220s 360ms/step - loss: 2.6016
Epoch 6/10
610/610 [==============================] - 218s 358ms/step - loss: 2.5008
Epoch 7/10
610/610 [==============================] - 218s 357ms/step - loss: 2.4064
Epoch 8/10
610/610 [==============================] - 217s 356ms/step - loss: 2.3206
Epoch 9/10
610/610 [==============================] - 218s 356ms/step - loss: 2.2416
Epoch 10/10
610/610 [==============================] - 217s 356ms/step - loss: 2.1663


In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


## 4. 학습모델 평가하기
* 모델이 작문을 잘하는지 평가하는 가장 확실한 방법은 작문을 시켜보고 직접 평가하는 것임  
* generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 함  

### 4-1. Test Loss

In [13]:
results = model.evaluate(enc_val, dec_val, verbose=2, batch_size=256)

print('test loss:', results)

123/123 - 16s - loss: 2.0865
test loss: 2.086527109146118


### 4-2. 작문 평가

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환함
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

**[Lyricist AI의 동작]** : 단어를 하나씩 예측해 문장을 만듬  
1. 입력받은 문장의 텐서를 입력함 
2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냄  
3. 위에서 예측된 word index를 문장 뒤에 붙임  
4. 모델이 \<end\> 를 예측했거나, max_len에 도달했다면 문장 생성을 마침  

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> yeh", max_len=20)

'<start> yeh i t wanna be with you <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> girl", max_len=20)

'<start> girl i m a voodoo chile <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> sky", max_len=20)

'<start> sky is the limit and you know that you can have <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m a motherfucking monster <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> this", max_len=20)

'<start> this is the only way <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> they", max_len=20)

'<start> they don t know what to do <end> '

### 4-3. 최종평가
**test_loss : 2.086527109146118  
작문실력 : 대체로 자연스럽게 문장을 잘 만들어냈음.**

---
## 5. 자가평가

언어모델을 처음 만들어보는데, 생각보다 문장을 너무 잘 생성해서 놀랐다.    
자연어처리는 CV와 학습 흐름이 많이 달랐는데, 토큰화된 데이터를 다층의 순환신경망에 넣어서 학습을 시켰다.   
이번에 사용한 RNN 은 요즘엔 잘 안쓰이는 모델인 것 같은데도 (더욱 성능 좋은 모델이 많기 때문에)  
성능이 만족스럽게 나왔다. 앞으로 더욱 발전된 형태의 모델을 사용하면 어떤 결과가 나올지 기대된다.     
  
이번 노드를 공부하면서 조원들과 "인공지능 가짜뉴스의 영향력"에 대한 토론을 했다.   
단순히 인간처럼 자연스럽게 언어를 사용하는 인공지능을 만드는 것을 목표로 하는 것이 아니라,  
이를 어떻게 활용할 것인지, 어떤 이로움이 있는지에 대한 고민이 필요하다고 느낀다.  

In [22]:
#- pandas 연습해보기
import pandas as pd
from pandas import DataFrame

data = {'평가사항': ['가사 텍스트 생성 모델이 정상적으로 동작하는가?','데이터 전처리와 데이터셋 구성 과정이 체계적으로 진행되었나?', '텍스트 생성모델이 안정적으로 학습되었나?'], '세부평가사항':['텍스트 제너레이션 결과가 그럴듯한 문장이 생성되는가?','특수문자 제거,토크나이저 생성, 패딩처리 등의 과정을 빠짐없이 수행했는가?','텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?'], 'answer': ['Yes', 'Yes', 'Yes']}
check = pd.DataFrame(data, columns = ['평가사항','세부평가사항','answer'], index = ['1.','2.','3.'])
check

,평가사항,세부평가사항,answer
1.,가사 텍스트 생성 모델이 정상적으로 동작하는가?,텍스트 제너레이션 결과가 그럴듯한 문장이 생성되는가?,Yes
2.,데이터 전처리와 데이터셋 구성 과정이 체계적으로 진행되었나?,"특수문자 제거,토크나이저 생성, 패딩처리 등의 과정을 빠짐없이 수행했는가?",Yes
3.,텍스트 생성모델이 안정적으로 학습되었나?,텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?,Yes
